In [3]:
import os
import sys

In [4]:
os.chdir("../")

In [5]:
%pwd

'u:\\nlp_project\\Image_Sharing_Plateform'

In [6]:
import os
import yaml
from ensure import ensure_annotations
from pathlib import Path
from typing import Any
import yaml
from types import SimpleNamespace
import string
from pickle import dump, load
from PIL import Image
import tensorflow as tf

In [5]:
import tqdm

In [6]:
def dict_to_namespace(d):
    """Recursively converts a dictionary into a SimpleNamespace object."""
    if isinstance(d, dict):
        return SimpleNamespace(**{k: dict_to_namespace(v) for k, v in d.items()})
    return d

In [7]:
import os
def create_directories(path_to_directories: list, verbose=True):
    for path in path_to_directories:
        os.makedirs(path, exist_ok=True)

def read_yaml(path_to_yaml: Path):
    try:
        with open(path_to_yaml, "r") as yaml_file:
            content = yaml.safe_load(yaml_file)
            return dict_to_namespace(content)  # Convert dict to namespace
    except FileNotFoundError:
        print(f"File not found: {path_to_yaml}")

In [8]:
def load_doc(filename):
    """
        For loading the document file and reading the contents inside the file into a string.
    """
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [9]:
from src.image_sharing_plateform.constants import *

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ExtractedFeatureConfig:
    image_data_path: Path
    extracted_feature_path: Path
    resize_img_height: int
    resize_img_width: int

In [11]:
config_filepath = CONFIG_FILE_PATH
config = read_yaml(config_filepath)

In [12]:
config.feature_extraction

namespace(image_data_path='U:/nlp_project/Image_Sharing_Plateform/data/row/Flickr8k_Dataset/Flicker8k_Dataset',
          extracted_feature_path='data/processed',
          resize_img_height=500,
          resize_img_width=500)

In [ ]:
class FeatureExtractionConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH):
        self.config = read_yaml(config_filepath)

    
    def get_feature_extraction_config(self) -> ExtractedFeatureConfig:
        config = self.config.feature_extraction

        # create_directories([config.extracted_feature_path])

        feature_extraction_config = ExtractedFeatureConfig(
            image_data_path = config.image_data_path,
            
            extracted_feature_path = config.extracted_feature_path,
            
            resize_img_height = config.resize_img_height,
            
            resize_img_width = config.resize_img_width
        )

        return feature_extraction_config

In [14]:
from tqdm.notebook import tqdm

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D
import numpy as np
import os
from tqdm import tqdm
from PIL import Image

class FeatureExtraction:
    def __init__(self, config):
        self.config = config
    
    def extract_features(self):
        image_data_path = self.config.image_data_path

        # Load VGG16 model without top layers
        base_model = VGG16(input_shape=(self.config.resize_img_height, self.config.resize_img_width, 3), 
                           include_top=False, weights='imagenet')

        # Add GlobalAveragePooling to convert (None, 15, 15, 512) → (None, 512)
        model = tf.keras.Sequential([
            base_model,
            GlobalAveragePooling2D()  # Converts spatial dimensions to (batch_size, 512)
        ])

        features = {}
        
        for img in tqdm(os.listdir(image_data_path)):
            filename = os.path.join(image_data_path, img)
            
            # Load and preprocess the image
            image = Image.open(filename)
            image = image.resize((self.config.resize_img_height, self.config.resize_img_width))
            image = np.array(image) / 255.0  # Normalize correctly
            image = np.expand_dims(image, axis=0)  # Add batch dimension
            
            # Extract feature and flatten it
            feature = model.predict(image)  # Shape: (1, 512)

            features[img] = feature.flatten()  # Convert (1, 512) → (512,)
        
        return features


In [ ]:
try:
    config = FeatureExtractionConfigurationManager()
    
    feature_extraction_config = config.get_feature_extraction_config()
    print(feature_extraction_config)
    
    data_transformation = FeatureExtraction(config=feature_extraction_config)
    
    extracted_features = data_transformation.extract_features()

except Exception as e:
    raise e

ExtractedFeatureConfig(image_data_path='U:/nlp_project/Image_Sharing_Plateform/data/row/Flickr8k_Dataset/Flicker8k_Dataset', extracted_feature_path='data/processed', resize_img_height=500, resize_img_width=500)
----------------------------------------------------------------------------------------------------------------
U:/nlp_project/Image_Sharing_Plateform/data/row/Flickr8k_Dataset/Flicker8k_Dataset
----------------------------------------------------------------------------------------------------------------


  0%|          | 0/8091 [00:00<?, ?it/s]

1/1 [==============================] - 9s 9s/step


  0%|          | 1/8091 [00:09<20:46:36,  9.25s/it]

1/1 [==============================] - 0s 18ms/step


...

100%|█████████▉| 8088/8091 [18:53<00:00, 10.08it/s]

1/1 [==============================] - 0s 20ms/step


100%|█████████▉| 8089/8091 [18:53<00:00,  9.83it/s]

1/1 [==============================] - 0s 20ms/step


100%|██████████| 8091/8091 [18:53<00:00,  7.14it/s]


In [8]:
features = load(open(r"U:\nlp_project\Image_Sharing_Plateform\data\processed\extracted_features\extracted_features.p","rb"))

In [9]:
features['1000268201_693b08cb0e.jpg'].shape

(512,)